Read CSV
Generate Table with headings
Generate y_train data
    Read Steering Angle
Generate X_train data
    Read images (center, left, right) using filename
    Crop
    Augment
        Flip,resize - append(y_train, offset)
        Noise, resize - append(y_train, offset)
        blur, resize - append(y_train, offset)

Neural Network Implmentation
training, validation, testing split
Generator Function

Neural Network Training
Save Model

    

### TUNING PARAMETERS

In [ ]:
# Steering Angle Offset
# Note: Steering Angle is normalized to -1 and 1
steering_ang_offset = 0.1

### Read CSV

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
csv_headers = ["center", "left", "right", "steering", "throttle", "brake", "speed"]

data = pd.read_csv('CarSim_data/driving_log.csv', names=csv_headers)

In [ ]:
# data.head()

### Features

In [ ]:
img_path = data['center'][1]
#print(img_path)
num_samples = len(data)
i=0
for row in data['center']:
#     print(i)
    I = cv2.imread(img_path)
    b,g,r = cv2.split(I)
    I = cv2.merge([r,g,b])
    #cv2.imwrite('I.jpg',I)
#     I = I[50:,:]
#     I = cv2.resize(I,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)
    plt.imshow(I)
    
    # https://carnd-forums.udacity.com/questions/32716893/
    # does-anyone-know-how-to-make-multiple-images-into-4d-arrays-like-pickled-data#
    #arr = np.zeros((num_examples, width, height, channels))

    X_train = np.zeros((num_samples, I.shape[0], I.shape[1], I.shape[2]))
    X_train[i, :, :, :] = I
    i=i+1

In [ ]:
print(type(I))
print(I.shape)

### Labels

In [ ]:
# Convert a pandas dataframe (df) to a numpy ndarray
y_train = data['steering'].values
print(y_train.shape)

### Create Features and Labels

### Preprocess the Data
1.) Shuffle the data

2.) Normalize the features using Min-Max scaling between -0.5 and 0.5

3.) Steering Angle: -25 to 25


In [ ]:
def i_crop(I):
    return I[50:,:]

def i_resize(I):
    return cv2.resize(I,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)

### Generator

In [ ]:
import sklearn
import pandas
from sklearn.utils import shuffle
import csv

In [ ]:
samples = []
with open('./CarSim_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

In [ ]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [ ]:
print(len(train_samples))

## Keras Model

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Convolution2D, Dense, Dropout, Flatten, Lambda, MaxPooling2D
from keras.optimizers import Adam
#from keras.layers.normalization import BatchNormalization


In [ ]:
# from keras.utils.visualize_util import plot

In [ ]:
ch, row, col = 3,320,110

In [ ]:
model = Sequential()

model.add(Lambda(lambda x: x/127.5-1., input_shape=(col, row, ch), output_shape=(col, row, ch)))


model.add(Convolution2D(24,5,5, init='glorot_uniform', subsample=(2, 2), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(36,5,5, init='glorot_uniform', subsample=(2, 2), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(48,5,5, init='glorot_uniform', subsample=(2, 2), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(64,3,3, init='glorot_uniform', border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(64,3,3, init='glorot_uniform', border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1164))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))

In [ ]:
#print(model.summary())

In [ ]:
model.compile(loss='mse', optimizer='adam')

In [ ]:
def myGenerator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # loop forever so generator never terminates
        shuffle(samples)
        # for logging
        batch_num_idx = 1
        for offset in range(0, num_samples, batch_size):
            
            print('Batch Number: ', batch_num_idx, ' End')
            print(' ')
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            
            for batch_sample in batch_samples:
                # Center Image
                name = batch_sample[0].strip()
                center_image = cv2.imread(name)
                center_image = i_crop(center_image)
                print(center_image.shape)
#                 center_image = i_resize(center_image)
                
                images.append(center_image)
#                 print('Length of Image List (Batch Sample Number): ', len(images))
                # Center Angle
                center_angle = float(batch_sample[3])
                angles.append(center_angle)

                # Left Image
                name = batch_sample[1].strip()
                left_image = cv2.imread(name)
                left_image = i_crop(left_image)
#                 left_image = i_resize(left_image)
                images.append(left_image)
                # Left Angle
                left_angle = float(batch_sample[3])-steering_ang_offset
                angles.append(left_angle)

                # Right Image
                name = batch_sample[2].strip()
                right_image = cv2.imread(name)
                right_image = i_crop(right_image)
#                 right_image = i_resize(right_image)
                images.append(right_image)
                plt.imshow(right_image)
                # Right Angle
                right_angle = float(batch_sample[3])+steering_ang_offset
                angles.append(right_angle)
                         
            X_train = np.array(images)
            print(X_train.shape)
            print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
            print(' ')
            y_train = np.array(angles)
            
            # for logging
            batch_num_idx += 1
            
            yield shuffle(X_train, y_train)

In [ ]:
# compile and train the model using generator function
train_generator = myGenerator(train_samples, batch_size=32)
validation_generator = myGenerator(validation_samples, batch_size=32)

In [ ]:
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=5)

### Save Model

In [ ]:
import json

model_json = model.to_json()
with open ('model.json', 'w') as f:
    json.dump(model_json, f, indent=4, sort_keys=True, separators=(',', ':'))
    
# model.save_weights will only save the weights
model.save('model.h5')
print("Model Saved")

In [ ]:
len(y_train)